# Introducción a la Estadística by M.Sc. Emerson D. Norabuena Figueroa
- Insdustria Car - Repositorio

## 1. Acerca del conjunto de datos

Contexto

El conjunto de datos mtcars es empleada a fin de informar de los atributos que fijan posibles métricas para el analista de datos - junior

Objetivo: Conocer los patrones a partir de la información recopilada para aprender y simular técnicas de análisis de datos sobre un entorno de desarrollo. 
Los atributos de esta base de datos son:

1. model: Modelo
2. mpg: Millas por galón
3. cyl: Número de cilindros
4. disp: Cilindrada en pulgadas cúbicas
5. hp: Caballos de fuerza
6. drat: Relación del eje de transmisión
7. wt: Peso
8. qsec: Tiempo de 1/4 de milla, medida de aceleración
9. vs: Forma del motor V o recto
10. am: Transmisión: 0: automático 1: manual
11. gear: Número de engranajes
12. carb: Número de carburadores

Contenido

1. Load and check data
2. Data visualization
3. Feature Engineering
4. Modeling Train - Test Split
5. Standarization





In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline

from statistics import mode
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

1. Load and check data

In [ ]:
datos_iniciales = pd.read_csv('https://raw.githubusercontent.com/Emerson2021/2023/main/mtcars.csv', sep=',')

In [ ]:
datos_iniciales.head()

In [ ]:
datos_iniciales.shape

In [ ]:
datos_iniciales.info

In [ ]:
print(sum(datos_iniciales.duplicated()))

In [ ]:
print(datos_iniciales.info())

In [ ]:
na_counts = pd.DataFrame(datos_iniciales.isna().sum(),columns=["NA Counts"]).reset_index()
na_counts = na_counts.rename(columns={'index': 'Column Name'})
print(na_counts)

In [ ]:
datos_iniciales.index = pd.RangeIndex(start=1,stop=33, step=1)
datos = datos_iniciales["mpg"]
minimo = min(datos)
maximo = max(datos)
rango = round(maximo - minimo,3)
criterio_sturges = 1+3.32*math.log10(len(datos))

print(f'El valor mínimo es de {minimo}, y el máximo de {maximo} mpg.')
print(f'Por lo que el rango es de {rango} mpg')
print(f'El número sugerido de columnas es de {round(criterio_sturges,2)}')

In [ ]:
numero_columnas = 6
minimo = math.floor(min(datos))
maximo = math.ceil(max(datos))
rango = round(maximo - minimo,3)
ancho_intervalo = rango/numero_columnas

j = 1
inicio_interval = []
final_interval = []

while j <= numero_columnas:
    inicio_interval.append(minimo + ancho_intervalo * (j-1))
    final_interval.append(minimo + ancho_intervalo * j)
    j = j + 1
    
tab_frecu = pd.DataFrame({"Inicio":inicio_interval,"Final":final_interval}).round(1)
tab_frecu['Inicio-Final'] = "["+tab_frecu["Inicio"].round(1).astype(str) + " - " + tab_frecu["Final"].round(1).astype(str)+")" 
tab_frecu

limite_bajo = tab_frecu["Inicio"]
limite_alto = tab_frecu["Final"]
valores = datos.copy()

frecuencias3 = []
fila = 0
while fila < len(limite_bajo):
    if fila == (len(limite_bajo)-1):
        x = (valores >= limite_bajo[fila]) & (valores <= limite_alto[fila])
        frecuencias3.append(len(valores[x]))
    else:
        x = (valores >= limite_bajo[fila]) & (valores < limite_alto[fila])
        frecuencias3.append(len(valores[x]))
    fila = fila + 1

tab_frecu["Frecuencia"] = frecuencias3
tab_frecu["Frecuencia Relativa(%)"] = ((tab_frecu["Frecuencia"]/tab_frecu["Frecuencia"].sum())*100).round(3)

fre_rel_acu3 = [] 
for p in range(1,tab_frecu["Inicio"].shape[0]+1):
    fre_rel_acu3.append(tab_frecu["Frecuencia Relativa(%)"].iloc[0:p].sum().round(2))

tab_frecu["Frecuencia Relativa Acumulada(%)"] = fre_rel_acu3
pd.options.display.float_format = '{:.1f}'.format
display(tab_frecu)

2. Data visualization

In [ ]:
plt.figure(figsize=(12,6))
bplot = datos_iniciales.boxplot(patch_artist=True)
plt.xticks(rotation=90)       
plt.show()

In [ ]:
datos_iniciales1 = datos_iniciales.sort_values("mpg").reset_index(drop=True) 
plt.rcParams.update({'font.size':7})
fig1, ax = plt.subplots(facecolor='white',figsize=(6,6)) 
fig1 = plt.barh(datos_iniciales1["model"],datos_iniciales1["mpg"],edgecolor='blue',color ="red")
ax.grid(axis='x',color = "white", linestyle = "--")
ax.set_xticks(np.arange(0, datos_iniciales1["mpg"].max(), 2.5))
plt.xlabel("Millas por galón")
plt.ylabel("Marca y modelo")
ax.set_xlim(right=34.2)
ax.set_ylim(bottom=-1,top=32)
plt.show()

In [ ]:
def graficador(datos,col,cal,col_2,cal_2,vlines):
    fig2, ax = plt.subplots(facecolor='white',figsize=(9,4)) 
    fig2 = plt.scatter(range(1,33),datos)
    if cal: 
        plt.plot(range(1,33),np.repeat(np.mean(datos),32),
                 label = f"media = {np.mean(datos).round(2)}",color = col ,linewidth = 2)
    if cal_2: 
        plt.plot(range(1,33),np.repeat(np.median(datos),32),
                 label = f"mediana = {np.median(datos).round(2)}",color = col_2 ,linewidth = 2)
    if vlines:
        ax.vlines(list(datos.index),datos,np.repeat(np.mean(datos),32),
              color = "green", linestyle = "dashed", linewidth=1.5,alpha = 0.75)
        
    ax.set_xticks(np.arange(1,33, 1))
    
    plt.ylabel("Millas por galón")
    plt.xlabel("Autos")
    plt.legend(loc='upper left')
    ax.set_xlim(left=0,right=32.5)
    plt.show()
    
graficador(datos_iniciales["mpg"],"red",True,"green",False,False)

In [ ]:
graficador(datos_iniciales["mpg"],"red",True,"green",False,True)


In [ ]:
datos = datos_iniciales["mpg"].sort_values()
fig2, ax = plt.subplots(facecolor='white',figsize=(9,4)) 
fig2 = plt.scatter(range(1,33),datos)
ax.vlines(16,9,np.median(datos_iniciales["mpg"]),
              color = "green", linestyle = "dashed", linewidth=1.5,alpha = 0.75)
ax.set_xticks(np.arange(1,33, 1))
plt.ylabel("Millas por galón")
plt.xlabel("Autos")
ax.set_xlim(left=0,right=32.5)
plt.ylim(9,35)
plt.show()

In [ ]:
fig2, ax = plt.subplots(facecolor='white',figsize=(9,4)) 
fig2 = plt.scatter(range(1,33),datos)
ax.vlines(16,9,np.median(datos),
              color = "green", linestyle = "dashed", linewidth=1.5,alpha = 0.75)
ax.set_xticks(np.arange(1,33, 1))
ax.vlines(17,9,np.median(datos),
              color = "green", linestyle = "dashed", linewidth=1.5,alpha = 0.75)
ax.set_xticks(np.arange(1,33, 1))

plt.ylabel("Millas por galón")
plt.xlabel("Autos")
plt.ylim(9,35)
ax.set_xlim(left=0,right=32.5)
plt.show()

In [ ]:
print("Valor de la posición 16:",datos.loc[16], "\nValor de la posición 17:",datos.loc[17])


In [ ]:
print("La mediana de los datos es:",np.median(datos))

In [ ]:
graficador(datos,"red",True,"green",True,False)

In [ ]:
ejeX = (tab_frecu["Inicio"] + tab_frecu["Final"])/2
ejeX2 = tab_frecu["Final"]
ejeY1 = tab_frecu["Frecuencia Relativa(%)"]
etiquetas1 = list(tab_frecu['Inicio-Final'])
titulo = "Distribución del consumo de combustible"
tituloX = "Millas por galon (mpg)"
tituloY = "Frecuencia relativa (%)"
intervalo = ancho_intervalo

fig_doble, axis = plt.subplots(1, 2,figsize=(14,5)) 
fig_doble.tight_layout(w_pad=2)


axis[0].bar(ejeX,ejeY1,width = intervalo,edgecolor='black',color='deepskyblue',alpha=0.80)
axis[0].vlines(np.mean(datos),0,35,label = f"media = {np.mean(datos).round(1)}",
              color = "red", linestyle = "dashed", linewidth=2 ,alpha = 1)
axis[0].vlines(np.median(datos),0,35,label = f"mediana = {np.median(datos).round(1)}",
              color = "lime", linestyle = (0,(1,1)), linewidth=3 ,alpha = 1)
axis[0].set_ylim(bottom=0,top=35)
axis[0].set_xlim(left=9.5,right=34.5)
axis[0].set_title(titulo)
axis[0].set_xlabel(tituloX)
axis[0].set_ylabel(tituloY)
axis[0].set_xticks(ejeX,etiquetas1)
axis[0].legend(bbox_to_anchor=(.65, 1), loc='upper left')


axis[1].bar(ejeX,ejeY1,width = intervalo,edgecolor='black',color='deepskyblue',alpha=0.80)
axis[1].vlines(np.mean(datos),0,35,label = f"media = {np.mean(datos).round(1)}",
              color = "red", linestyle = "dashed", linewidth=2 ,alpha = 1)
axis[1].vlines(np.median(datos),0,35,label = f"mediana = {np.median(datos).round(1)}",
              color = "lime", linestyle = (0,(1,1)), linewidth=3 ,alpha = 1)
axis[1].set_ylim(bottom=0,top=35)
axis[1].set_xlim(left=9.5,right=34.5)
axis[1].set_title(titulo)
axis[1].set_xlabel(tituloX)
axis[1].set_ylabel(tituloY)
axis[1].set_xticks(ejeX2)
axis[1].legend(bbox_to_anchor=(.65, 1), loc='upper left')
plt.show()

In [ ]:
ejeX = (tab_frecu["Inicio"]+tab_frecu["Final"])/2
ejeX2 = tab_frecu["Final"]
ejeY2 = tab_frecu["Frecuencia Relativa Acumulada(%)"]

tituloX = "Millas por galon (mpg)"
tituloY = "Frecuencia relativa acumulada (%)"
intervalo = ancho_intervalo
etiquetas2 = list(range(0,125,25))

fig_doble, axis = plt.subplots(1, 2,figsize=(14,5)) 
fig_doble.tight_layout(w_pad=2)

# Grafico izquierda
axis[0].bar(ejeX,ejeY2,width = intervalo,edgecolor='black',color='deepskyblue',alpha=0.80)
axis[0].vlines(np.mean(datos),0,105,label = f"media = {np.mean(datos).round(1)}",
              color = "red", linestyle = "dashed", linewidth=2 ,alpha = 1)
axis[0].vlines(np.median(datos),0,105,label = f"mediana = {np.median(datos).round(1)}",
              color = "lime", linestyle = (0,(1,1)), linewidth=3 ,alpha = 1)


axis[0].set_ylim(bottom=0,top=105)
axis[0].set_xlim(left=9.5,right=34.5)
axis[0].set_title(F'{titulo} (histograma)')
axis[0].set_xlabel(tituloX)
axis[0].set_ylabel(tituloY)
axis[0].set_xticks(ejeX2)
axis[0].legend(bbox_to_anchor=(0.3, 1), loc='upper right')


punto = axis[1].plot(ejeX2,ejeY2,color = "black",linewidth = 1.5)
axis[1].plot(ejeX2,ejeY2,'o', markersize=8,color = "darkorange")

for x, y in zip(ejeX2, ejeY2):
    axis[1].text(x+.35,y-3,'{:.1f}%'.format(y))

axis[1].vlines(np.mean(datos),0,105,label = f"media = {np.mean(datos).round(1)}",
              color = "red", linestyle = "dashed", linewidth=2 ,alpha = 1)
axis[1].vlines(np.median(datos),0,50,label = f"mediana = {np.median(datos).round(1)}",
              color = "lime", linestyle = (0,(1,1)), linewidth=3 ,alpha = 1)
axis[1].hlines(50,0,np.median(datos),color = "lime", linestyle = (0,(1,1)), linewidth=3 ,alpha = 1)

axis[1].set_ylim(bottom=0,top=105)
axis[1].set_xlim(left=13.5,right=36.5)
axis[1].set_title(F'{titulo} (gráfico de línea)')
axis[1].set_xlabel(tituloX)
axis[1].set_ylabel(tituloY)
axis[1].set_xticks(ejeX2)
axis[1].set_yticks(etiquetas2)
axis[1].legend(bbox_to_anchor=(1, 0), loc='lower right')
plt.show()